In [1]:
# import os
# from bs4 import BeautifulSoup, NavigableString, Tag 

from nltk.util import ngrams
from nltk import word_tokenize

# import json
import numpy as np
import pandas as pd

%matplotlib inline
import matplotlib as plt
import seaborn as sns

import collections
import csv
# from sets import Set

from datetime import datetime
from pytz import timezone

maxInt = sys.maxsize

while True:
    # decrease the maxInt value by factor 10 
    # as long as the OverflowError occurs.

    try:
        csv.field_size_limit(maxInt)
        break
    except OverflowError:
        maxInt = int(maxInt/10)

print("imported")

imported


In [2]:
# IMPORTS SPECIFIC TO TOPIC MODELING

# import langid
import nltk
import re
import time
from collections import defaultdict
from configparser import ConfigParser
from gensim import corpora, models, similarities
from nltk.tokenize import RegexpTokenizer
from string import digits
from nltk.stem import WordNetLemmatizer
print("imported")

imported


import editorial text

In [3]:
import import_func as imp
import tags_work as tgs
import desc_vis as vis
import classifier_func as cls
import classifier_help as clh

### Loading Documents and Splitting Into Lines

In [4]:
# full_advice = "../data/genre_advice_full_021520.jl"
full_filename = "../data/by_article_fulltext_020920.jl"

# get full text dataset as a df
# advice_df = imp.init_df(full_advice, "full", genre="advice")
# advice_df = advice_df[advice_df["year"]<2020]
full_df = imp.init_df(full_filename, "full")
full_df = full_df[full_df["year"]<2020]

full_df.head(3)

,id,headline,tags,authors,date,time,text,bio,date_seq,month_seq,year,author,n_posts_author,column1,column2,advice,one_time
0,5839,how to write a winning résumé,"[tooling up, advice, graduate, academic, indus...",peter fiske,1996-10-18,8:00 am,"by \nwelcome to ""tooling up,"" a monthly colu...",[peter fiske is a ph.d. scientist and co-found...,18,10,1996,peter fiske,59,no,yes,yes,no
1,5835,the commandments of cover letter creation,"[tooling up, advice, americas]",peter fiske,1996-12-20,0:00 am,"by \n\n've always hated the term ""cover lett...",[peter fiske is a ph.d. scientist and co-found...,81,12,1996,peter fiske,59,no,yes,yes,no
2,5840,dressing scientists for success: male case study,"[tooling up, advice, graduate, postdoc, academ...",peter fiske,1997-09-26,8:00 am,"by \n\ne all know that, with respect to fash...",[peter fiske is a ph.d. scientist and co-found...,361,21,1997,peter fiske,59,no,yes,yes,no


In [5]:
#LOAD DOCUMENT BY LINE INTO AN ARRAY

# Need to make separate docs for each unit of analysis


# files = [filename.split('.')[0] for filename in os.listdir('digitalSTS/papers/txt/') if not filename.startswith('.')]
# print files

line_num = [i for i in range(len(full_df))]
section = ["advice","news","wl"]

####################
### dictionaries ###
####################

all_words = []

year_words = {}
for i in range(1996,2020):
    year_words[str(i)] = []
    # print(i)

sect_words = {}
for j in section:
    sect_words[j] = []

sect_year_words = {}
for i in range(1996,2020):
    for j in section:
        sect_year_words[str(j)+"_"+str(i)] = []

for num in line_num:
    # outpath = 'digitalSTS/papers/lines/' + filename + '.csv'
    # w_test = full_df.iloc[num]["text"]
    # with open(outpath, 'w') as outfile:
        
#         for line in infile.readlines():
#             print line
    paragraphs = full_df.iloc[num]["text"]
    # print(paragraphs)
    year = full_df.iloc[num]["year"]
    if full_df.iloc[num]["advice"]=="yes":
        sect="advice"
    elif full_df.iloc[num]["one_time"]=="yes":
        sect="wl"
    else:
        sect="news"

    for paragraph in paragraphs.split("\n"):
        # print("=====")
        # print(paragraph)
        if paragraph == "" or paragraph.strip() == " " or paragraph.strip() == "by":
            continue

        # paragraph = paragraph.decode('unicode_escape').encode('ascii','ignore')
        lines = [line.strip().replace("--"," ") for line in re.split(".?!",paragraph)]#''.join(paragraph).split('.')]
        #             print paragraph
        
        # writer = csv.writer(outfile)

        for line in lines:
            # print("#####")
            # print(line)
            # it may be tempting to seek out a bargain discount men's clothing store, but you tend to get what you pay for
            # line = line.replace("-"," ").replace("―"," ").replace("ē","e").replace(" "," ").replace("ﬁ","fi").replace("β","beta").replace("►","").replace("‐"," ")
            words = re.split(" ",line)

            if len(words) > 1:
                # print(words)
                all_words.append(line)
                year_words[str(year)].append(line)
                sect_words[sect].append(line)
                sect_year_words[str(sect)+"_"+str(year)].append(line)
                # full2df["id"].append(full_df.iloc[num]["id"])
                # full2df["line_words"].append(words)
                # full2df["type"].append(ty)

                #     writer.writerow([line.encode('utf-8')])
# print(len(year_words["2010"]))
# print(len(sect_words["advice"]))
# print(sect_words["news"][:3])

In [7]:
outfilename = "../data/dictionary_files/all_text/txts/all_text.txt"
with open(outfilename, "w",encoding='utf-8') as outfile:
    for line in all_words:
        line += "\n"
        # line = line.encode('utf-8')
        # print(line)
        outfile.write(line)


for key in sect_words.keys():
    outfilename = "../data/dictionary_files/section/txts/"+key+".txt"
    with open(outfilename, "w",encoding='utf-8') as outfile:
        for line in sect_words[key]:
            line += "\n"
            # line = line.encode('utf-8')
            # print(line)
            outfile.write(line)

for key in year_words.keys():
    outfilename = "../data/dictionary_files/year/txts/"+key+".txt"
    with open(outfilename, "w",encoding='utf-8') as outfile:
        for line in year_words[key]:
            line += "\n"
            # line = line.encode('utf-8')
            # print(line)
            outfile.write(line)

for key in sect_year_words.keys():
    outfilename = "../data/dictionary_files/section_year/txts/"+key+".txt"
    with open(outfilename, "w",encoding='utf-8') as outfile:
        for line in sect_year_words[key]:
            line += "\n"
            # line = line.encode('utf-8')
            # print(line)
            outfile.write(line)

In [21]:
files = [filename.split('.')[0] for filename in os.listdir('../data/dictionary_files/sec/txts/') if not filename.startswith('.')]
print(files)

['1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019']


### Creating Dictionaries and Corpuses

In [9]:


files = [filename.split('.')[0] for filename in os.listdir('../data/dictionary_files/all_text/txts/') if not filename.startswith('.')]
# print files

for filename in files:

    inpath = '../data/dictionary_files/all_text/txts/' + filename + '.txt'
    print(inpath)

    with open(inpath, 'r',encoding='utf-8') as infile:
        
        reader = csv.reader(infile)
        documents = [line[0] for line in reader]
#         print lines

        # TOKENIZING
        tokenizer = RegexpTokenizer(r'\w+')
        documents = [tokenizer.tokenize(doc.lower()) for doc in documents ]
        
        # GATHERING CANDIDATES FOR CUSTOM LIST OF STOPWORDS FROM NGRAMS
        # ['i','m','a','a','i','t','6',...]
        unigrams = [ w for doc in documents for w in doc if len(w)==1]
        # ['do','or','or','of','be',...]
        bigrams  = [ w for doc in documents for w in doc if len(w)==2]

        misc = ['get', 'know', 'would']
        
        # CREATES THE SET OF UNIQUE "CUSTOM STOP LIST WITH NLTK + THE ABOVE
        stoplist  = set(nltk.corpus.stopwords.words("english") + unigrams + bigrams + misc)

        # REMOVES STOP WORDS
        documents = [[token for token in doc if token not in stoplist]
                        for doc in documents]

        # REMOVES WORDS THAT ARE NUMBERS ONLY
        documents = [ [token for token in doc if len(token.strip(digits)) == len(token)]
                        for doc in documents ]
        
        # REMOVE WORDS THAT OCCUR ONLY ONCE
        token_frequency = defaultdict(int)

        # count all tokens
        for doc in documents:
            for token in doc:
                token_frequency[token] += 1

        # keep words that occur more than once
        documents = [ [token for token in doc if token_frequency[token] > 1]
                        for doc in documents  ]

        # Sort words in documents
        for doc in documents:
            doc.sort()
            
        # # HATE UNICODE!!
        # # cuments = [ [token.dor token in doc] for doc in documents  ]
  #        
        # Build a dictionary where for each document each word has its own id
        dictionary = corpora.Dictionary(documents)
        dictionary.compactify()
        # and save the dictionary for future use
 
        print(dictionary)
        dictionary.save( '../data/dictionary_files/all_text/dicts/' + filename + '.dict')

      
        # BUILD AND SAVE CORPUS
        corpus = [dictionary.doc2bow(doc) for doc in documents]
 
        corpora.MmCorpus.serialize('../data/dictionary_files/all_text/corpuses/' + filename + '.mm', corpus)




../data/dictionary_files/all_text/txts/all_text.txt
Dictionary(25409 unique tokens: ['tooling', 'welcome', 'applicant', 'clearly', 'goals']...)


In [10]:
# CODE HELP FROM:
# https://github.com/alexperrier/datatalks/blob/master/twitter/twitter_preprocessing.py

# SPLITS EACH POST INTO WORDS WHILE REMOVING PUNCTUATION (\w+) AND LOWER CASES THEM A WORD SUCH AS
# I'M WILL BECOME I, M
# test = [[doc for doc in docu] for docu in documents ]
# print("hi")
# print(test[:3])
documents_one = [" ".join(doc) for doc in documents]
documents = documents_one

tokenizer = RegexpTokenizer(r'\w+')
documents = [tokenizer.tokenize(doc.lower()) for doc in documents ]
## [['what', 'do', 'they', 'look', 'for', 'for', 'approval', 'or', 'denial'],
##  ['hi','i','m','wondering','if','someone','could','answer',...]...
# documents[:2]

In [11]:
# I'M NOT SURE THESE ARE TECHNICALLY UNIGRAMS AND BIGRAMS SO MUCH AS THEY ARE SINGLE LETTER WORDS
# AND TWO LETTER WORDS BUT WHO AM I TO JUDEGE (SHULDER SHURG)

# ['i','m','a','a','i','t','6',...]
unigrams = [ w for doc in documents for w in doc if len(w)==1]
# ['do','or','or','of','be',...]
bigrams  = [ w for doc in documents for w in doc if len(w)==2]

misc = ['get', 'know', 'would']

print(unigrams[:5])
print(bigrams[:5])

[]
[]


In [12]:
# CREATES THE SET OF UNIQUE "CUSTOM STOP LIST WITH NLTK + THE ABOVE
stoplist  = set(nltk.corpus.stopwords.words("english") + unigrams + bigrams + misc)

# REMOVES STOP WORDS
documents = [[token for token in doc if token not in stoplist]
                for doc in documents]

# REMOVES WORDS THAT ARE NUMBERS ONLY
documents = [ [token for token in doc if len(token.strip(digits)) == len(token)]
                for doc in documents ]

# # [['look', 'approval', 'denial', 'parole', 'approval'...],
# # ['wondering', 'someone', 'could', 'answer', 'question'...]...]
print(documents[0][:5])
print(documents[1][:5])

['tooling', 'welcome']
['applicant', 'clearly', 'goals', 'states']


In [13]:
# REMOVE WORDS THAT OCCUR ONLY ONCE
token_frequency = defaultdict(int)

# count all tokens
for doc in documents:
    for token in doc:
        token_frequency[token] += 1

# keep words that occur more than once
documents = [ [token for token in doc if token_frequency[token] > 1]
                for doc in documents  ]

# Sort words in documents
for doc in documents:
    doc.sort()
    
## [['approval', 'approval', 'approval', 'approval', 'approval'...],
##  ['answer', 'benefit', 'benefit', 'benefit', 'confused'...]...]

print(documents[0][:5])
print(documents[1][:5])

['tooling', 'welcome']
['applicant', 'clearly', 'goals', 'states']


In [36]:
# I HATE UNICODE!!
# documents = [ [token.decode('ascii', 'ignore') for token in doc] for doc in documents  ]
print(documents[0][:5])
print(documents[1][:5])

['academics', 'career', 'cheese', 'colleagues', 'early']
['dollars', 'enter', 'health', 'make', 'public']


In [14]:
# Build a dictionary where for each document each word has its own id
dictionary = corpora.Dictionary(documents)
dictionary.compactify()
# and save the dictionary for future use
dictionary.save('all_text.dict')

In [15]:
# We now have a dictionary with 11127 unique tokens
# Dictionary(11127 unique tokens: [u'foul', u'narcotic', u'four', u'woods', u'hanging']...)
print(dictionary)

Dictionary(25409 unique tokens: ['tooling', 'welcome', 'applicant', 'clearly', 'goals']...)


In [16]:
# Build the corpus: vectors with occurence of each word for each document
# convert tokenized documents to vectors
corpus = [dictionary.doc2bow(doc) for doc in documents]

In [17]:
# and save in Market Matrix format
corpora.MmCorpus.serialize('all_text.mm', corpus)
# this corpus can be loaded with corpus = corpora.MmCorpus('prison_talk_nyc.mm')

## MISC

### Lemmatization

In [48]:
files = [filename.split('.')[0] for filename in os.listdir('../data/dictionary_files/years/txts/') if not filename.startswith('.')]
# files

FileNotFoundError: [WinError 3] The system cannot find the path specified: '../data/dictionary_files/years/txts/'

In [47]:
# filename = '02_Parmiggiani_Monteiro'
# testpath = '../papers/tests/' + '02_Parmiggiani_Monteiro' + '.csv'

totalfq = collections.Counter()

for filename in files:

    inpath = '../data/dictionary_files/years/txts/' + filename + '.txt'
    outpath = '.../data/dictionary_files/years/lemmas/' + filename + '.txt'
    
    with open(inpath, 'r') as infile, open(outpath, 'w') as outfile:

        essay = infile.readline()

        tokenizer = RegexpTokenizer(r'\w+')
        tokens = tokenizer.tokenize(essay.lower())

        # ['i','m','a','a','i','t','6',...]
        unichars = [word for word in tokens if len(word)==1]
        # ['do','or','or','of','be',...]
        bichars  =  [word for word in tokens if len(word)==2]

        misc = ['get', 'know', 'would', 'and', 'yet', 'due', 'one', 'digital', 'technology', 'way', \
                'also', 'use', 'within', 'like', 'sts', 'may', 'however', 'even', u'technology', 'used']

        stoplist  = set(nltk.corpus.stopwords.words("english") + unichars + bichars + misc)

        # REMOVES WORDS THAT ARE NUMBERS ONLY
        post_stop = [token for token in tokens if len(token.strip(digits)) == len(token)]

        # REMOVES STOP WORDS
        post_stop = [token for token in tokens if token not in stoplist]

        post_lemma = [lemma.lemmatize(token) for token in post_stop]

        tokenfq = collections.Counter()
        tokenfq.update(post_lemma)
        
        post_highfq = [token for token in post_lemma if tokenfq[token] > 1]

        totalfq.update(post_highfq)
        
        writer = csv.writer(outfile)
        writer.writerow([' '.join(post_highfq).encode('utf-8')])

        print(filename), '----> TOKENS IN:', len(tokens), 'POST STOP:', len(post_stop), 'POST HIGHFQ:', len(post_highfq), \
                        '----> PCT:', round(float(len(post_highfq))/float(len(tokens)), 2) * 100

FileNotFoundError: [Errno 2] No such file or directory: '../data/dictionary_files/years/txts/advice.txt'